In [1]:
import rdflib
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON

# Load Word Embeddings

In [2]:
import logging
from six import iteritems
from web.embeddings import load_embedding, fetch_conceptnet_numberbatch
from web.evaluate import evaluate_similarity, evaluate_similarity_score

In [3]:
%%time
#Fetch FastText-wiki 300d 1M
we =load_embedding("C:\\Users\\daix8\\web_data\\embeddings\\fastText\\wiki-news-300d-1M.vec\\wiki-news-300d-1M.vec", format='word2vec', normalize=True, lower=False, clean_words=False)

Wall time: 3min 49s


In [41]:
%%time
#Fetch ConceptNetNumb
wecn = fetch_conceptnet_numberbatch()

File already downloaded, skipping
Wall time: 1min 37s


In [4]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# isEntityInDatabase Method

In [5]:
def isEntityInDatabase(s):
    return s.startswith("http")

# MakeQueryString Method

In [6]:
def makeQueryString(obj, filterStringsPredicates, filterStringSubjects, literals):
    if obj in literals:
        result = "SELECT distinct ?s ?p WHERE { " + "{?s ?p " + obj + " . " + "} FILTER ("
        for s in filterStringsPredicates:
            fsp = "!regex(str(?p), '" + s + "' , 'i') && "
            result += fsp
        for s in filterStringSubjects:
            fss = "!regex(str(?s), '" + s + "' , 'i') && "
            result += fss
        result = result[:result.rindex("&&")] + ")}"
    else:
        result = "SELECT distinct ?s ?p ?j ?k WHERE { "
        incomingQuery = "{?s ?p <" + obj + "> . FILTER ("
        outgoingQuery =  "{<" + obj + "> ?j ?k. FILTER ("
        
        for s in filterStringsPredicates:
            fsp = "!regex(str(?p), '" + s + "' , 'i') && "
            incomingQuery += fsp
            fsj = "!regex(str(?j), '" + s + "' , 'i') && "
            outgoingQuery += fsj
            
        for s in filterStringSubjects:
            fss = "!regex(str(?s), '" + s + "' , 'i') && "
            fss2 = "!regex(str(?k), '" + s + "' , 'i') && "
            incomingQuery += fss
            outgoingQuery += fss2
        
        incomingQuery = incomingQuery[:incomingQuery.rindex("&&")] + ")}"
        outgoingQuery = outgoingQuery[:outgoingQuery.rindex("&&")] + ")}"
        result = result + incomingQuery + " UNION " + outgoingQuery + "}"
    return result

# Triple Class 

In [7]:
class Triple:
    def __init__(self):
        self.subject = None
        self.object = None
        self.predicate = None
        self.cost = None
        self.previousTriple = None
        #the list of connecting seeds
        self.seeds = []
    
    def __str__(self):
        if(self.subject is not None and self.predicate is not None and self.object is not None):
            # print connecting seeds
            connectSeeds = "["
            for seed in self.seeds:
                connectSeeds += seed + ", "
            connectSeeds = connectSeeds[:connectSeeds.rindex(',')] + "]"
            return self.subject + "  --  " + self.predicate + "  --  " + self.object + "  " + connectSeeds + " (" + str(self.cost) + ')'
        else:
            return "Not Well Defined Triple"
        
    def __eq__(self,other):
        if(self.subject == other.getSubject() and self.object == other.getObject() and self.predicate == other.getPredicate()):
            return True
        else:
            return False
        
    def __gt__(self,other):
        if self.cost > other.getCost():
            return True
        else:
            return False
    
    def setSeeds(self, fs):
        if fs not in self.seeds:
            self.seeds.append(fs)
            self.seeds.sort()
            
    def getSeeds(self):
        return self.seeds
    
    def setPreviousTriple(self, pt):
        self.previousTriple = pt
        
    def getPreviousTriple(self):
        return self.previousTriple
    
    def updateSeeds(self, newSeeds):
        for seed in newSeeds:
            self.setSeeds(seed)
            
    def setSubject(self, sub):
        self.subject = sub
    def setObject(self, obj):
        self.object = obj
    def setPredicate(self, pre):
        self.predicate = pre
    def setCost(self, c):
        self.cost = c
    def getCost(self):
        return self.cost
    def getSubject(self):
        return self.subject
    def getObject(self):
        return self.object
    def getPredicate(self):
        return self.predicate

# Expand Method

In [8]:
'''
def expand(queryTriples, eg, di, predicatesToMatch, wemb):
    newQueryTriples = []
    matchingTriples = []
    
    tmp = []
    print("queryTriple size : " + str(len(queryTriples)))
    for queryObject in queryTriples:
        queryString = makeQueryString(queryObject.getObject(), filterStringPredicates,filterStringSubjects, literalsToConnect)
        print("----- Current Query ------")
        print(queryString)
        print("--------------------------")
        
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            if 's' in result:
                if(isEntityInDatabase(result["s"]["value"])):
                    #print(result["s"]["value"] + "   ---   " + result["p"]["value"])
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["p"]["value"])
                    newTriple.setObject(result["s"]["value"])
                    #newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["p"]["value"], wemb ))
                    #newTriple.setCost(20)
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    #print(newTriple)
                    
                    #Apply word embedding for costs and modify matchingTriples
                    if newTriple not in matchingTriples:
                        matchingTriples.append(newTriple)
                    
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                    
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
            else:
                if(isEntityInDatabase(result["k"]["value"])):
                    #print(result["k"]["value"] + "   ---   " + result["j"]["value"])
                    
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["j"]["value"])                
                    newTriple.setObject(result["k"]["value"])
                    #newTriple.setCost(20)
                    #newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["j"]["value"], wemb ))
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    
                    #print(newTriple)
                    
                    #Apply word embedding for costs and modify matchingTriples
                    if newTriple not in matchingTriples:
                        matchingTriples.append(newTriple)
                    
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                        
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
        computeCosts(predicatesToMatch, matchingTriples, wemb)
    return newQueryTriples, matchingTriples
'''
print("Old version of expand")

Old version of expand


In [9]:
def expand(queryTriples, eg, di, predicatesToMatch, wemb):
    newQueryTriples = []
    matchingTriples = []
    
    tmp = []
    print("queryTriple size : " + str(len(queryTriples)))
    for queryObject in queryTriples:
        queryString = makeQueryString(queryObject.getObject(), filterStringPredicates,filterStringSubjects, literalsToConnect)
        print("----- Current Query ------")
        print(queryString)
        print("--------------------------")
        
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            
            if 's' in result:
                newTriple = Triple()
                newTriple.setSubject(queryObject.getObject())
                newTriple.setPredicate(result["p"]["value"])
                newTriple.setObject(result["s"]["value"])
                newTriple.setPreviousTriple(queryObject)
                    
                for seed in queryObject.getSeeds():
                    newTriple.setSeeds(seed)
                    
                if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                
                #Apply word embedding for costs and modify matchingTriples
                if newTriple not in matchingTriples:
                    matchingTriples.append(newTriple)                       
                
                if(isEntityInDatabase(result["s"]["value"])):                                   
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                    
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
                        
            else:
                                    
                newTriple = Triple()
                newTriple.setSubject(queryObject.getObject())
                newTriple.setPredicate(result["j"]["value"])                
                newTriple.setObject(result["k"]["value"])
                newTriple.setPreviousTriple(queryObject)

                for seed in queryObject.getSeeds():
                    newTriple.setSeeds(seed)

                #Apply word embedding for costs and modify matchingTriples
                if newTriple not in matchingTriples:
                    matchingTriples.append(newTriple)

                if newTriple not in expandedGraph:
                    expandedGraph.append(newTriple)
                    
                if(isEntityInDatabase(result["k"]["value"])):
                    #print(result["k"]["value"] + "   ---   " + result["j"]["value"])

                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                        
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
        computeCosts(predicatesToMatch, matchingTriples, wemb)
    return newQueryTriples, matchingTriples

# addDuplicatedItems Method

In [10]:
def addDuplicatedItems(ntp, eg, di):
    for tp in eg:
        if tp.getObject() == ntp.getObject() and tp not in di:
            di.append(tp)
            if tp.getSeeds() != ntp.getSeeds():
                updateSeedsInExpandedGraph(tp,ntp,eg)           
    #di.append(ntp)
    return "Dup"

In [11]:
def updateSeedsInExpandedGraph(tp,ntp,eg):
    old1 = tp.getSeeds()[:]
    old2 = ntp.getSeeds()[:]
    ntp.updateSeeds(tp.getSeeds())
    newS = ntp.getSeeds()
    
    for triple in eg:
        if triple.getSeeds() == old1 or triple.getSeeds() == old2:
            triple.updateSeeds(newS)

# Reduction Tests

In [12]:
def reductionTestsDegreeOne(eg):
    
    degreeOneNodes = []
    
    for tp1 in eg:
        duplicatedObject = False
        isLeafNode = True
        for tp2 in eg:
            if tp2.getPreviousTriple() == tp1:
                isLeafNode = False
            if tp1.getObject() == tp2.getObject() and not tp1.getSubject() == tp2.getSubject():
                duplicatedObject = True
                break
        if not duplicatedObject and isLeafNode:
            degreeOneNodes.append(tp1)
    print("Degree One Nodes size: " + str(len(degreeOneNodes)))
    for tp in degreeOneNodes:
        eg.remove(tp)

#TODO: Fix Bug
def keepMinEdge(eg):
    
    triplesToBeRemoved = []
    
    for tp1 in eg:
        minTriple = tp1
        for tp2 in eg:
            if tp1.getSubject() == tp2.getSubject() and tp1.getObject() == tp2.getObject and not tp1.getPredicate() == tp2.getPredicate():
                if tp2.getCost() < tp1.getCost():
                    minTriple = tp2
                    if tp1 not in triplesToBeRemoved:
                        triplesToBeRemoved.append(tp1)
                else:
                    if tp2 not in triplesToBeRemoved:
                        triplesToBeRemoved.append(tp2)
    for tp in triplesToBeRemoved:
        eg.remove(tp)
    

# checkConnection Method

In [13]:
def checkConnection(ltc, di):
    if len(ltc) == 1:
        return False
    else:
        ltc.sort()
        for tp in di:
            if tp.getSeeds() == ltc:
                return True
        return False

# ComputeCosts Method

In [46]:
def computeCosts(predicatesToMatch, matchingTriples, wemb):
    th = 30
    predicateList = []
    
    for tp in matchingTriples:
        predicate = tp.getPredicate()
        predicateList.append(predicate[predicate.rindex('/')+1:])
        
    for y in predicatesToMatch:
        ar = []
        for p in predicateList:
            ar.append([y, p])
        ar = np.array(ar)
        result = evaluate_similarity_score(wemb, ar, 'xxx')
        #print("size of result :" + str(len(result)) + '  ----  size of matching Triples : ' + str(len(matchingTriples)))
        for x in range(len(result)):
            #when two comparing predicates are both not in the word embedding, then using JW distance instead
            #TODO: What if one of them is in word embedding?
            if result[x] == 1.0 and y != predicateList[x]:
                #print(y + "  ---  " + predicateList[x])
                result[x] = dw(y, predicateList[x])
            elif (1 - result[x]) * 100 > th:
                result[x] = dw(y, predicateList[x])
                
            if matchingTriples[x].getCost() is None:
                matchingTriples[x].setCost((1 - result[x]) * 100)
            elif matchingTriples[x].getCost() > (1 - result[x]) * 100:
                matchingTriples[x].setCost((1 - result[x]) * 100)
    
    index = 0
    
    while index < len(matchingTriples):
        if matchingTriples[index].getCost() > th:
            matchingTriples.pop(index)
        else:
            index += 1


In [15]:
def computeCosts2(predicatesToMatch, matchingTriples, wemb):
    th = 30
    predicateList = []
    
    for tp in matchingTriples:
        predicate = tp.getPredicate()
        predicateList.append(predicate[predicate.rindex('/')+1:])
        
    for y in predicatesToMatch:
        ar = []
        for p in predicateList:
            ar.append([y, p])
        ar = np.array(ar)
        result = evaluate_similarity_score(wemb, ar, 'xxx')
        #print("size of result :" + str(len(result)) + '  ----  size of matching Triples : ' + str(len(matchingTriples)))
        for x in range(len(result)):
            if matchingTriples[x].getCost() is None:
                matchingTriples[x].setCost((1 - result[x]) * 100)
            elif matchingTriples[x].getCost() > (1 - result[x]) * 100:
                matchingTriples[x].setCost((1 - result[x]) * 100)
    
    index = 0
    
    while index < len(matchingTriples):
        if matchingTriples[index].getCost() > th:
            matchingTriples.pop(index)
        else:
            index += 1


# Identify Entities and predicates in keywords
###### 1. check if the lowercased literal is a property in dataset. If yes, then it is a predicate.
###### 2. check if the convertFirstToCapital literal is a property in dataset. If yes, then it is a predicate.
###### 3. check if the convertFirstToCapital literal is a resource in dataset. If yes, then it is an entity
###### 4. Treat every literal as a predicate.

In [23]:
def isResourceInDataset(literal):
    jw_th = 0.7
    
    queryString = 'select distinct ?s where {?s ?p "'+ literal.lower() + '"@en}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        #print(result["s"]["value"])
        sub = result["s"]["value"]
        if 'property' in sub:
            if dw(sub[sub.rindex('/')+1:],literal) > jw_th:
                return False
    
    literal = convertFirstToCapital(literal)
    
    queryString = 'select distinct ?s where {?s ?p "'+ literal + '"@en}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    propertyInSub = False
    resourceInSub = False
    for result in results["results"]["bindings"]:
        #print(result["s"]["value"])
        sub = result["s"]["value"]
        p = dw(sub[sub.rindex('/')+1:],literal)
        if 'property' in sub:
            if  p > jw_th:
                propertyInSub = True
        elif 'resource' in sub:
            if p > jw_th:
                resourceInSub = True
                
    if propertyInSub:
        return False
    elif resourceInSub:
        return True
    else:
        return False

In [17]:
def indentifyEntitiesAndPredicates(keywords):
    entity = []
    pre = []
    
    li = keywords.split(',')
    
    for x in range(len(li)):
        li[x] = li[x].strip()
        if isResourceInDataset(li[x]):
            entity.append(convertFirstToCapital(li[x]))
        # we treat every keyword as a predicate
        pre.append(li[x])
    
    for x in range(len(entity)):
        entity[x] = '"' + entity[x] + '"@en'
    return entity,pre


# Jaro_Winkler Distance

In [18]:
def make1shorter(word1, word2):
    if len(word1) > len(word2):
        # Make sure word1 is shorter
        temp = word1
        word1 = word2
        word2 = temp

    return word1, word2

def dj(word1, word2):
    if len(word1) == 0 or len(word2) == 0:
        raise Exception("Not words, mate")

    word1, word2 = make1shorter(word1, word2)

    word2chars = list(word2)
    m = 0
    for char in word1:
        if char in word2chars:
            m += 1
            word2chars.pop(word2chars.index(char))

    t = 0
    for i in range(len(word1)):
        if word1[i] != word2[i]:
            t += 1

    if m == 0:
        return 0
    return 1/3*(m/len(word1) + m/len(word2) + (m - t / 2)/m)

def dw(word1, word2, p=0.1, lmax=4):
    word1, word2 = make1shorter(word1, word2)

    dj_ = dj(word1, word2)

    l = 0
    for i in range(min(len(word1), lmax)):
        if word1[i] == word2[i]:
            l += 1
        else:
            break

    return dj_ + l * p * (1 - dj_)

# Convert first letter in literals to capital case and lower case for the rest

In [19]:
def convertFirstToCapital(s):
    res = s.split()
    for x in range(len(res)):
        res[x] = res[x][0].upper() + res[x][1:].lower()
    res = " ".join(res)
    return res

# Main Program

In [44]:
%%time

with open('qald5-1.txt','r',encoding='utf-8') as f:
    lines = f.readlines()

with open('test_res.txt','w',encoding='utf-8') as fw:
    for line in lines:
        filterStringPredicates = ["wikiPageWikiLink","wikiPageRedirects","wikiPageDisambiguates", "Thing","wikiPageUsesTemplate","rdf-syntax-ns#type"]
        filterStringSubjects = ["entity", "Category", "wikidata","owl#Thing", "http://wikidata.dbpedia.org/resource/Q"]
        expandedGraph = []
        duplicatedItems = []

        literalsToConnect, predicates = indentifyEntitiesAndPredicates(line.strip())
        fw.write("-----Question------\n")
        fw.write(line + '\n')
        fw.write("-------------------\n")
        qtps = []
        for literal in literalsToConnect:
            triple = Triple()
            triple.setObject(literal)
            triple.setSeeds(literal)
            qtps.append(triple)

        fw.write('--entities--\n')
        print('---- entities ----')
        for tp in qtps:
            fw.write(str(tp.getObject()) + '\n')
            print(tp.getObject())
        print(predicates)
        # Test Purpose

        levelOfExpansion = 1
        while(levelOfExpansion < 3 and not checkConnection(literalsToConnect, duplicatedItems)):
            #add mtps to a new list
            qtps, mtps = expand(qtps, expandedGraph, duplicatedItems, predicates, we)
            levelOfExpansion += 1

        print("Matching Triples: -----   size: " + str(len(mtps)))
        mtps.sort()
        for tp in mtps:
            fw.write(str(tp) + '\n')
            print(tp)
        if checkConnection(literalsToConnect, duplicatedItems):
            reductionTestsDegreeOne(expandedGraph)
            fw.write("Size of expanded graph: "+ str(len(expandedGraph)) + '\n')
            for tp in expandedGraph:
                fw.write(str(tp) + '\n')

---- entities ----
"Alberta"@en
['Alberta', 'admit', 'province']
queryTriple size : 1
----- Current Query ------
SELECT distinct ?s ?p WHERE { {?s ?p "Alberta"@en . } FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )}
--------------------------


Missing 34 words. Will replace them with mean vector
Missing 34 words. Will replace them with mean vector
Missing 34 words. Will replace them with mean vector


queryTriple size : 37
----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Alberta> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Alberta> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(st

Missing 3568 words. Will replace them with mean vector
Missing 3568 words. Will replace them with mean vector
Missing 3568 words. Will replace them with mean vector


----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Alberta_(singer)> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Alberta_(singer)> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(str(?k

...

http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Bruce_Dickson  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Christie_Mjolsness  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Clyde_Gilmour  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Donald_Gauf  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Eric_Paterson  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Frank_Pierpoint_Appleby  ["Alberta"@en] (37.539684772491455)
htt

http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Jordon_Cooke  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Kelly_Davis  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Marco_Carducci  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Mark_Benning  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Mitch_Holmberg  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Paul_Cranmer  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/

http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Steve_Bell_(musician)  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Trevor_Devall  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Chelsey_Reist  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Kiesza  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Michelle_Salt  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Sonia_Arrison  ["Alberta"@en] (37.539684772491455)
http://dbpedia.

http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Devon_Weigel  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Dutch_Gainor  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Jeff_Finley  ["Alberta"@en] (37.539684772491455)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/birthPlace  --  http://dbpedia.org/resource/Jeff_Glass  ["Alberta"@en] (37.539684772491455)


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Test Above

In [42]:
%%time
levelOfExpansion = 1
while(levelOfExpansion < 3 or not checkConnection(literalsToConnect, duplicatedItems)):
    #add mtps to a new list
    qtps, mtps = expand(qtps, expandedGraph, duplicatedItems, predicates, we)
    levelOfExpansion += 1

print("Matching Triples: -----   size: " + str(len(mtps)))
for tp in mtps:
    print(tp)

queryTriple size : 9764
----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Senate_of_Berlin> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Senate_of_Berlin> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity'

Missing 33 words. Will replace them with mean vector
Missing 33 words. Will replace them with mean vector


----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Berlin> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Berlin> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(str(?k), 'Category' , 'i')

Missing 7742 words. Will replace them with mean vector
Missing 7742 words. Will replace them with mean vector


----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Brandenburg> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Brandenburg> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(str(?k), 'Catego

...

Missing 75 words. Will replace them with mean vector
Missing 75 words. Will replace them with mean vector


----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Daniel_Chodowiecki> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Daniel_Chodowiecki> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(st

Missing 92 words. Will replace them with mean vector
Missing 92 words. Will replace them with mean vector


----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Hildegard_Knef> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Hildegard_Knef> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(str(?k), '

KeyboardInterrupt: 

In [42]:
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

Size of expanded graph: 546
"Ganges"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Ganges  ["Ganges"@en] (69.66674327850342)
"Ganges"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Ganges_(1792_ship)  ["Ganges"@en] (59.351956844329834)
"Ganges"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Ganges,_Hérault  ["Ganges"@en] (59.351956844329834)
"Ganges"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Ganges_(1798_ship)  ["Ganges"@en] (59.351956844329834)
"Ganges"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Ganges_(1861)  ["Ganges"@en] (59.351956844329834)
"Ganges"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Ganges_(1882)  ["Ganges"@en] (59.351956844329834)
"Ganges"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Ganges_(BBC_TV_series)  ["Ganges"@en] (59.351956844329834)
"Ganges"@en  --  http://xmlns.

*  Ganges, en inglés El río nace en el Himalaya occidental, en el estado indio de Uttarakhand, y tras 2510 km de recorrido desemboca formando, con el río Brahmaputra el mayor delta del mundo, el delta del Ganges, en el golfo de Bengala.  ["Ganges"@en] (58.42236578464508)
http://dbpedia.org/resource/Ganges  --  http://www.w3.org/2000/01/rdf-schema#comment  --  Il Gange (chiamato localmente Ganga, Devanagari गंगा) è un grande fiume del subcontinente indiano che scorre verso oriente attraversando le pianure del nord dell'India e il Bangladesh. Ha una lunghezza di 2.510 km e le sue sorgenti sono localizzate sul ghiacciaio di Gangotri nello stato indiano dell'Uttarakhand, nell'Himalaya centrale. Sfocia nel Golfo del Bengala con un ampio delta nella regione del Sundarbans. Per millenni ha goduto di una posizione preminente nella religione indù in India ed è adorato nella sua forma personificata della dea Ganga.  ["Ganges"@en] (58.42236578464508)
http://dbpedia.org/resource/Ganges  --  http:/

http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/abstract  --  الغانج (بالهندية: गंगा، وتنطق "غانغا"، بالبنغالية:গঙ্গা، وتنطق "غونغا"، بالإنجليزية: The Ganges) هو أحد أكبر أنهار شبه القارة الهندية. يجري باتجاه الشرق مخترقاً السهل الغانغي (بالإنجليزية: Gangetic Plain) في شمال الهند وينتهي في بنغلاديش. يبلغ طوله 2510 كيلومتراً (1560 ميلاً) وينبع من جبال الهيمالايا الغربية في ولاية أوتاراخند الهندية وينتهي بدلتا مليئة بالغابات قرب مصبه في خليج البنغال. تبلغ مساحة حوض النهر حوالي 907.000 كم². وينظر الهندوس إلى نهر الغانغ باعتباره نهراً مقدساً ويعبدون إلهاً يمثله هو الإله غانغا. وقد تمتع نهر الغانغ بأهمية تاريخية مرجعها وقوع العديد من عواصم الهند الإقليمية أو الإمبراطورية على ضفافه (مثل باتاليبوترا (بالإنجليزية: Pataliputra) وقنوج وكاره (بالإنجليزية: Kara) والله أباد ومرشد أباد وكلكتا). ويبلغ متوسط عمق النهر 52 قدماً (16 متراً) وأقصى عمق له 100 قدم (30 متراً). وقد أعلنت الهند نهر الغانغ نهرها القومي. من حيث الطول هو الخامس عشر في آسيا وال39 في العالم. ينبع من جبال الهملاي

http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/Haridwar  ["Ganges"@en] (61.89093887805939)
http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/Patna  ["Ganges"@en] (61.89093887805939)
http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/Bhagalpur  ["Ganges"@en] (61.89093887805939)
http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/Buxar  ["Ganges"@en] (61.89093887805939)
http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/Allahabad  ["Ganges"@en] (61.89093887805939)
http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/country  --  http://dbpedia.org/resource/Bangladesh  ["Ganges"@en] (62.78031766414642)
http://dbpedia.org/resource/Ganges  --  http://dbpedia.org/ontology/country  --  http://dbpedia.org

http://dbpedia.org/resource/Ganges_(1792_ship)  --  http://dbpedia.org/property/shipOwner  --  Thomas Patrickson  ["Ganges"@en] (67.43589639663696)
http://dbpedia.org/resource/Ganges_(1792_ship)  --  http://dbpedia.org/property/shipArmament  --  12  ["Ganges"@en] (68.9615398645401)
http://dbpedia.org/resource/Ganges_(1792_ship)  --  http://dbpedia.org/property/shipComplement  --  80  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1792_ship)  --  http://dbpedia.org/property/shipCountry  --  United Kingdom  ["Ganges"@en] (46.18381857872009)
http://dbpedia.org/resource/Ganges_(1792_ship)  --  http://dbpedia.org/property/shipLaunched  --  1792  ["Ganges"@en] (68.9615398645401)
http://dbpedia.org/resource/Ganges_(1792_ship)  --  http://dbpedia.org/property/shipNamesake  --  http://dbpedia.org/resource/Ganges  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1792_ship)  --  http://dbpedia.org/property/shipTonsBurthen  --  617  ["Ganges"@en] (55.560

http://dbpedia.org/resource/Ganges_(1798_ship)  --  http://www.w3.org/2000/01/rdf-schema#comment  --  Ganges was a ship launched in 1798 at Philadelphia probably for French owners. During the Peace of Amiens her registration and homeport became Dunkirk. Her (possibly new) French owners sent her to engage in whaling at Delagoa Bay, where the British letter of marque whaler Scorpion captured her in 1803. She then made one whaling voyage to Isle of Desolation before a French squadron captured her in 1806 during a second whaling voyage. Accounts differ as to whether her captors sank her, or released her and she continued to operate as a merchant vessel until 1814.  ["Ganges"@en] (58.42236578464508)
http://dbpedia.org/resource/Ganges_(1798_ship)  --  http://dbpedia.org/ontology/wikiPageID  --  51499060  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1798_ship)  --  http://dbpedia.org/ontology/wikiPageRevisionID  --  738777129  ["Ganges"@en] (69.66674327850342)
http://

http://dbpedia.org/resource/Ganges_(1882)  --  http://dbpedia.org/ontology/status  --  Sunk 28 May 1917  ["Ganges"@en] (64.85842168331146)
http://dbpedia.org/resource/Ganges_(1882)  --  http://dbpedia.org/property/hideHeader  --  no  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1882)  --  http://dbpedia.org/property/shipAcquired  --  1904  ["Ganges"@en] (68.9615398645401)
http://dbpedia.org/resource/Ganges_(1882)  --  http://dbpedia.org/property/shipCountry  --  Norway  ["Ganges"@en] (46.18381857872009)
http://dbpedia.org/resource/Ganges_(1882)  --  http://dbpedia.org/property/shipCountry  --  United Kingdom  ["Ganges"@en] (46.18381857872009)
http://dbpedia.org/resource/Ganges_(1882)  --  http://dbpedia.org/property/shipRenamed  --  Asters  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1882)  --  http://dbpedia.org/property/shipTonsBurthen  --  1768  ["Ganges"@en] (55.56043982505798)
http://dbpedia.org/resource/Ganges_(1882)  --  http://

http://dbpedia.org/resource/Ganges_(1794_ship)  --  http://dbpedia.org/property/shipLaunched  --  1794  ["Ganges"@en] (68.9615398645401)
http://dbpedia.org/resource/Ganges_(1794_ship)  --  http://dbpedia.org/property/shipNamesake  --  http://dbpedia.org/resource/Ganges  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1794_ship)  --  http://dbpedia.org/property/shipSailPlan  --  http://dbpedia.org/resource/Brig  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1794_ship)  --  http://dbpedia.org/property/shipTonsBurthen  --  130  ["Ganges"@en] (55.56043982505798)
http://dbpedia.org/resource/Ganges_(1794_ship)  --  http://purl.org/voc/vrank#hasRank  --  nodeID://b17150776  ["Ganges"@en] (69.66674327850342)
http://dbpedia.org/resource/Ganges_(1794_ship)  --  http://purl.org/linguistics/gold/hypernym  --  http://dbpedia.org/resource/Brig  ["Ganges"@en] (46.15244269371033)
http://dbpedia.org/resource/Ganges_(1797_ship)  --  http://xmlns.com/foaf/0.1

In [103]:
print("Size of duplicatedItems: "+ str(len(duplicatedItems)))
for tp in duplicatedItems:
    print(tp)

Size of duplicatedItems: 4
http://dbpedia.org/resource/Hells_Angels  --  http://xmlns.com/foaf/0.1/primaryTopic  --  http://en.wikipedia.org/wiki/Hells_Angels  ["Hells Angels"@en] (70.74291408061981)
http://dbpedia.org/resource/Hells_Angels  --  http://xmlns.com/foaf/0.1/isPrimaryTopicOf  --  http://en.wikipedia.org/wiki/Hells_Angels  ["Hells Angels"@en] (70.74291408061981)
http://dbpedia.org/resource/Hells_Angels  --  http://dbpedia.org/ontology/knownFor  --  http://dbpedia.org/resource/Sonny_Barger  ["Hells Angels"@en] (70.74291408061981)
http://dbpedia.org/resource/Hells_Angels  --  http://dbpedia.org/property/keyPeople  --  http://dbpedia.org/resource/Sonny_Barger  ["Hells Angels"@en] (70.74291408061981)


In [20]:
reductionTestsDegreeOne(expandedGraph)
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

Degree One Nodes size: 597
Size of expanded graph: 9
"Viking Press"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Viking_Press  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
"Jack Kerouac"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Jack_Kerouac  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/On_the_Road  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Door_Wide_Open  ["Jack Kerouac"@en, "Viking Press"@en] (51.2871056

In [20]:
keepMinEdge(expandedGraph)
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

Size of expanded graph: 0


# Test Purpose -- Main

In [45]:
k = np.array([['mayor', 'city']])
result = evaluate_similarity_score(wecn, k, 'xxx')
print(result[0])

0.5351437


In [46]:
w1 = 'admit'
w2 = 'admit'
print(dw(w1,w2))

1.0


In [33]:
print('1' != '2')

True


In [48]:
dw('province','admitancedate')

0.2692307692307692

In [51]:
filterStringPredicates = ["wikiPageWikiLink","wikiPageRedirects","wikiPageDisambiguates", "Thing","wikiPageUsesTemplate","rdf-syntax-ns#type"]
filterStringSubjects = ["entity", "Category", "wikidata","owl#Thing", "http://wikidata.dbpedia.org/resource/Q"]
expandedGraph = []
duplicatedItems = []
line = 'Alberta, admit, province'
literalsToConnect, predicates = indentifyEntitiesAndPredicates(line.strip())
qtps = []
for literal in literalsToConnect:
    triple = Triple()
    triple.setObject(literal)
    triple.setSeeds(literal)
    qtps.append(triple)

for tp in qtps:
    print(tp.getObject())

# Test Purpose

levelOfExpansion = 1
while(levelOfExpansion < 3 and not checkConnection(literalsToConnect, duplicatedItems)):
    #add mtps to a new list
    qtps, mtps = expand(qtps, expandedGraph, duplicatedItems, predicates, we)
    levelOfExpansion += 1

print("Matching Triples: -----   size: " + str(len(mtps)))
for tp in mtps:
    print(tp)

"Alberta"@en
queryTriple size : 1
----- Current Query ------
SELECT distinct ?s ?p WHERE { {?s ?p "Alberta"@en . } FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )}
--------------------------


Missing 34 words. Will replace them with mean vector
Missing 34 words. Will replace them with mean vector


queryTriple size : 37
----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Alberta> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Alberta> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(st

Missing 3568 words. Will replace them with mean vector
Missing 3568 words. Will replace them with mean vector


---------------
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/admittancedate  --  1905-09-01  ["Alberta"@en] (12.857145071029663)
-------------------
---------------
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/admittanceorder  --  11  ["Alberta"@en] (13.333332538604736)
-------------------
----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Alberta_(singer)> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Alberta_(sing

...

Missing 63 words. Will replace them with mean vector
Missing 63 words. Will replace them with mean vector


---------------
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/admittancedate  --  1905-09-01  ["Alberta"@en] (12.857145071029663)
-------------------
---------------
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/admittanceorder  --  11  ["Alberta"@en] (13.333332538604736)
-------------------
Matching Triples: -----   size: 359
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CJLI  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CJNW-FM  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CKEA-FM  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CKNO-FM  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resou

http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CJEG-FM  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CJGY-FM  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CKDQ  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CKIK-FM  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/CKMH-FM  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  --  http://dbpedia.org/resource/Calgary_Jr._A_Mountaineers  ["Alberta"@en] (38.92361521720886)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/ontology/city  -- 

http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/province  --  http://dbpedia.org/resource/University_of_Alberta_Campus_Saint-Jean  ["Alberta"@en] (0.0)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/province  --  http://dbpedia.org/resource/Husky_Lloydminster_Refinery  ["Alberta"@en] (0.0)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/region  --  http://dbpedia.org/resource/Kananaskis_Range  ["Alberta"@en] (35.12551188468933)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/region  --  http://dbpedia.org/resource/Rainbow_Range_(Rocky_Mountains)  ["Alberta"@en] (35.12551188468933)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/region  --  http://dbpedia.org/resource/Bow_Range  ["Alberta"@en] (35.12551188468933)
http://dbpedia.org/resource/Alberta  --  http://dbpedia.org/property/region  --  http://dbpedia.org/resource/Crowsnest_Range  ["Alberta"@en] (35.12551188468933)
http://dbpe

http://dbpedia.org/resource/Alberta_Williams_King  --  http://www.w3.org/ns/prov#wasDerivedFrom  --  http://en.wikipedia.org/wiki/Alberta_Williams_King?oldid=744915200  ["Alberta"@en] (25.350874662399292)
http://dbpedia.org/resource/Alberta_of_Agen  --  http://purl.org/dc/terms/description  --  Martyr and saint  ["Alberta"@en] (35.33549904823303)
http://dbpedia.org/resource/Alberta_of_Agen  --  http://www.w3.org/ns/prov#wasDerivedFrom  --  http://en.wikipedia.org/wiki/Alberta_of_Agen?oldid=707325407  ["Alberta"@en] (25.350874662399292)
http://dbpedia.org/resource/Alberta_Jeannette_Cassell  --  http://purl.org/dc/terms/description  --  American architect  ["Alberta"@en] (35.33549904823303)
http://dbpedia.org/resource/Alberta_Jeannette_Cassell  --  http://www.w3.org/ns/prov#wasDerivedFrom  --  http://en.wikipedia.org/wiki/Alberta_Jeannette_Cassell?oldid=736105081  ["Alberta"@en] (25.350874662399292)
http://dbpedia.org/resource/Alberta_Jeannette_Cassell  --  http://dbpedia.org/property/pr

In [25]:
print(isResourceInDataset("wrote"))

True


In [104]:
queryString = 'select distinct ?s ?p where {<http://dbpedia.org/resource/The_Hunger_Games> ?p ?s}'
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["p"]["value"] + " --- " +result["s"]["value"])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://www.w3.org/2002/07/owl#Thing
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/ontology/TelevisionShow
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/WikicatScienceFictionNovelTrilogies
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/Abstraction100002137
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/Arrangement107938773
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/Artifact100021939
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/Book106410904
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/Collection107951464
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/Communication100033020
http://www.w3.org/1999/02/22-rdf-syntax-ns#type --- http://dbpedia.org/class/yago/Creation

http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/North_America
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Revelation
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Rocky_Mountains
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Science_fiction
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Action_(fiction)
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Adventure_novel
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/E-book
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Category:The_Hunger_Games
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Liam_Hemsworth
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Reality_TV
http://dbpedia.org/ontology/wikiPageWikiLink --- http://dbpedia.org/resource/Medi